In [ ]:
class ActorCritic(nn.Module):
   def __init__(self, state_dim, action_dim, hidden_dim=128):
       super(ActorCritic, self).__init__()

       self.actor = nn.Sequential(
           nn.Linear(state_dim, hidden_dim),
           nn.ReLU(),
           nn.Linear(hidden_dim, hidden_dim),
           nn.ReLU(),
           nn.Linear(hidden_dim, action_dim)
       )
       self.critic =  nn.Sequential(
           nn.Linear(state_dim, hidden_dim),
           nn.ReLU(),
           nn.Linear(hidden_dim, hidden_dim),
           nn.ReLU(),
           nn.Linear(hidden_dim, 1)
       )

In [ ]:
   def forward(self, x):
       policy_logits = self.actor(x)
       value = self.critic(x)
       return policy_logits, value

   def act(self, state):
       """Выбирает действие и возвращает log_prob, предсказанную ценность и энтропию."""
       state_tensor = torch.from_numpy(state).float().unsqueeze(0)
       policy_logits, value = self(state_tensor)

       dist = Categorical(logits=policy_logits)
       action = dist.sample()
       log_prob = dist.log_prob(action)
       entropy = dist.entropy().mean()

       return action.item(), log_prob, value, entropy

In [ ]:
# Инициализируем переменные для сбора данных в течение N шагов
log_probs_buffer = []
values_buffer = []
rewards_buffer = []

state, _ = env.reset()
done = False
episode_reward = 0
episode_count = 0

while episode_count < n_episodes:
   # Собираем данные в буфер до тех пор, пока не наберется N шагов или пока эпизод не закончится
   for t in range(n_steps):
       if done:
           # Если эпизод закончился, сбрасываем среду
           episode_reward = 0
           episode_count += 1
           state, _ = env.reset()

       # Определяем действие с помощью модели
       action, log_prob, value = model.act(state)
       # Производим действие в среде
       next_state, reward, done, _, _ = env.step(action)
       episode_reward += reward
       # Запоминаем данные для вычисления обновления
       log_probs_buffer.append(log_prob)
       values_buffer.append(value)
       rewards_buffer.append(reward)

       state = next_state

In [ ]:
# Если эпизод завершился, next_value = 0.
# В противном случае, получаем ценность следующего состояния из сети.
if done:
   next_value = torch.tensor(0.0).float()
else:
   with torch.no_grad():
       next_state_tensor = torch.from_numpy(state).float().unsqueeze(0)
       _, next_value = model(next_state_tensor)
       next_value = next_value.squeeze()

In [ ]:
td_targets = torch.zeros(len(rewards_buffer))
# Начинаем с ценности следующего состояния (с дисконтом)
discounted_return = next_value
# Идем по наградам в обратном порядке
for t in reversed(range(len(rewards_buffer))):
   discounted_return = rewards_buffer[t] + gamma * discounted_return
   td_targets[t] = discounted_return

In [ ]:
# Объединяем список из собранных предсказаний критика в тензор
values_tensor = torch.cat(values_buffer).squeeze()
# Потери Критика
critic_loss = nn.MSELoss()(values_tensor, td_targets.detach())

In [ ]:
# Вычисляем преимущества (advantages) - это разница между возвратом и предсказанием
advantages = td_targets - values_tensor
# Потери Актора
actor_loss = -(log_probs_tensor * advantages.detach()).mean()
# Энтропийный бонус (опционально)
actor_loss -= с_entropy * entropies_tensor